# Biodiversity data - Codecademy Portfolio Project


### Import Python Libraries

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from tabulate import tabulate

## Data file species_info.csv

### Load

In [14]:
species_info = pd.read_csv('species_info.csv')
print(tabulate(species_info.head(), headers='keys', tablefmt='pretty'))
# observations = pd.read_csv('observations.csv')
# observations.head()

+---+----------+-------------------------------+----------------------------------------------------------------+---------------------+
|   | category |        scientific_name        |                          common_names                          | conservation_status |
+---+----------+-------------------------------+----------------------------------------------------------------+---------------------+
| 0 |  Mammal  | Clethrionomys gapperi gapperi |                    Gapper's Red-Backed Vole                    |         nan         |
| 1 |  Mammal  |           Bos bison           |                     American Bison, Bison                      |         nan         |
| 2 |  Mammal  |          Bos taurus           | Aurochs, Aurochs, Domestic Cattle (Feral), Domesticated Cattle |         nan         |
| 3 |  Mammal  |          Ovis aries           |       Domestic Sheep, Mouflon, Red Sheep, Sheep (Feral)        |         nan         |
| 4 |  Mammal  |        Cervus elaphus         |